## AWS Client Setup
Installa la AWS CLI scaricandola da questo url: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#getting-started-install-instructions

Una volta installata, incolla le informazioni dalla Command line or programmatic access nella folder .aws nel profilo utente di sistema.


Nel file config
```json
[username]
region = us-west-2
output = json
```

Nel file credential inserire le informazioni in questo formato:

```json
[username]
aws_access_key_id=
aws_secret_access_key=
aws_session_token=
```

In [163]:

import os
from utils import bedrock
from langchain.llms import Bedrock


os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "[fillthis]"

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.3}
    )

Create new client
  Using region: us-west-2
  Using profile: sky
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


# Prompt Engineering
Prompt engineering è una disciplina relativamente nuova che consente di sviluppare e ottimizzare i prompt per utilizzare in modo efficiente i modelli linguistici (LM) per un'ampia varietà di applicazioni e argomenti di ricerca. Le competenze di ingegneria dei prompt aiutano a comprendere meglio le capacità e i limiti dei modelli linguistici di grandi dimensioni (LLM).

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# Zero Shot
Gli Instruct LLM sono addestrati già di base per seguire le istruzioni. Spesso senza iniettare alcuna informazione ulteriore sono già in grado di comportarsi bene e rispondere alle nostre richieste

In [11]:
template = """Classify the input text in one of the following class: ['negative','neutral','positive'].
Input: {text}
Sentiment: 
"""

prompt = ChatPromptTemplate.from_template(template=template)
chain = prompt | model

In [12]:
chain.invoke({"text":"the trainer is really bad. I'm in the worse shape of my life"})

' Okay, let\'s analyze this:\n\nThe input text contains several negative phrases like "the trainer is really bad" and "I\'m in the worse shape of my life". These suggest the overall sentiment is negative.\n\nTherefore, I would classify this text as \'negative\'.'

Se vogliamo solo la classe in output dobbiamo essere più espliciti

In [13]:
template = """Classify the input text in one of the following class: ['negative','neutral','positive'].
Just give me the sentiment in output and nothing else.
Input: {text}
Sentiment: 
"""

prompt = ChatPromptTemplate.from_template(template=template)
chain = prompt | model

In [14]:
chain.invoke({"text":"the trainer is really bad. I'm in the worse shape of my life"})

' negative'

Voglio comunque sfruttare il ragionamento che il modello ha fatto, con una catena sequenziale

In [19]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.globals import set_debug

set_debug(True)

In [36]:
template_1 = """Classify the input text in one of the following class: ['negative','neutral','positive'].
Input: {text}
Think step by step
Sentiment: 
"""

template_2 = """Given the following explanation given in output by an LLM, classify the input text in one of the following class: ['negative','neutral','positive'].
Just give me the sentiment in output and nothing else.
Input: {reasoning}
Output: 
"""

prompt_1 = ChatPromptTemplate.from_template(template=template_1)
chain_1 = prompt_1 | model

prompt_2 = ChatPromptTemplate.from_template(template=template_2)
chain_2 = ({"reasoning":chain_1} | prompt_2 | model)


In [37]:
chain_1.invoke({"text":"you are on average"})

' Okay, let\'s break this down step-by-step:\n\n1. The input text is: "you are on average"\n\n2. This is a simple statement without any obvious sentiment words. Words like "great", "terrible", "love", "hate" etc would indicate positive or negative sentiment.\n\n3. The lack of any clear sentiment words makes this a neutral statement. \n\n4. Therefore, I would classify this text as \'neutral\'.\n\nSentiment: neutral'

In [22]:
chain_2.invoke({"text":"you are on average"},config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "text": "you are on average"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<reasoning>] Entering Chain run with input:
{
  "text": "you are on average"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<reasoning> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "text": "you are on average"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<reasoning> > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "text": "you are on average"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<reasoning> > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor

' neutral'

### Branching and Merging

Possiamo combinare le catene in varie sequenze, per ottenere output creativi e completi.

In [29]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter
from langchain.globals import set_debug

set_debug(False)

In [100]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | {"base_response": RunnablePassthrough()}
)
arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | model
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | model
)
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Combine pros and cons in an unique statement and make a final summary"),
        ]
    )
    | model
)
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [101]:
chain.invoke({"input": "USA - Democrat vs Republican "})

' Here is a balanced summary integrating the pros and cons of political differences between Democrats and Republicans in the United States:\n\nThe partisan divide between Democrats and Republicans has both advantages and disadvantages for American democracy. On the positive side, having two major competing parties with divergent viewpoints encourages robust debate of critical issues and provides citizens with distinct choices at the ballot box. The clash of ideological perspectives can fuel compromise, spur innovation, and prevent any one faction from dominating. \n\nHowever, stark political polarization also carries risks. Party conflicts can lead to gridlock in government, inflamed rhetoric on social issues, and disincentives for cooperation. Partisan media exaggerates differences while special interests drive parties to ideological extremes. Gerrymandering of districts contributes to the divide. Identity politics can breed resentment between groups.\n\nOverall, it seems the healthie

# Few Shot
Il few shot consente di migliorare e indirizzare il comportamento del modello nel risolvere particolari task fornendogli alcuni esempi. 

In [106]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.globals import set_debug

set_debug(False)

In [107]:
examples = [
    {"input": "You are ugly", "output": "1"},
    {"input": "look, it's raining ", "output": "3"},
    {"input": "I love you", "output": "5"},
    {"input": "I like the way you build this model. Well done!", "output": "5"},
    {"input": "You are not even able to do this work for me. You are useless", "output": "1"},
    {"input": "You did an ok job", "output": "4"},
    {"input": "this is not that great, next time do better please", "output": "2"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("assistant", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [108]:
few_shot_prompt.format()

"Human: You are ugly\nAI: 1\nHuman: look, it's raining \nAI: 3\nHuman: I love you\nAI: 5\nHuman: I like the way you build this model. Well done!\nAI: 5\nHuman: You are not even able to do this work for me. You are useless\nAI: 1\nHuman: You did an ok job\nAI: 4\nHuman: this is not that great, next time do better please\nAI: 2"

In [209]:
messages = [
            ("system", "You are a sentiment analysis assistant that give a sentiment between 1 and 5, where 5 means extremely positive and 1 extremely negative. Respect the format in the following example"),
            few_shot_prompt,
            ("human", "{text}"),
            ("assistant", ""),
        ]

prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [210]:
chain.invoke({"text":"I don't like the color of this t-shirt but could be worse"})

c:\Users\EGRIMAAY3\OneDrive - NTT DATA EMEAL\Desktop\antonio\progetti\2023\sky\workshop\aws-workshop-genai\.venv\Lib\site-packages\langchain_community\llms\bedrock.py:73: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received System: You are a sentiment analysis assistant that give a sentiment between 1 and 5, where 5 means extremely positive and 1 extremely negative. Respect the format in the following example

Human: You are ugly
AI: 1

Human: look, it's raining 
AI: 3

Human: I love you
AI: 5

Human: I like the way you build this model. Well done!
AI: 5

Human: You are not even able to do this work for me. You are useless
AI: 1

Human: You did an ok job
AI: 4

Human: this is not that great, next time do better please
AI: 2

Human: I don't like the color of this t-shirt but could be worse
AI: 

Assistant:
  warnings.warn(ALTERNATION_ERROR + f" Received {input_text}")


ValueError: Error raised by bedrock service: An error occurred (ExpiredTokenException) when calling the InvokeModel operation: The security token included in the request is expired

# Chain-Of-Thought

Un'altra tecnica molto nota è quella della Catena del pensiero. Portando il modello a ragionare invece di dare la risposta secca, è stato dimostrato che la probabilità di ottenere una risposta corretta è più alta. Nei modelli moderni la Chain of Thought è incorporata direttamente nella logica dei modelli. Noterete che assegnando a questo modello un problema logico matematico molto spesso iniziarà da solo la generazione con la dicitura "let\'s go through this step-by-step"

In [148]:
template = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

In [149]:
model.invoke(template)

' Okay, let\'s go through this step-by-step:\n\nThe odd numbers in the group are: 5, 13, 7, 1\nAdding them up: \n5 + 13 + 7 + 1 = 26\n\n26 is an even number.\n\nTherefore, the statement "The odd numbers in this group add up to an even number" is True for this set of numbers.'

# Self Consistency

Gli LLM possono sbagliare, e anzi lo fanno spesso. Una tecnica per verificare la veridicità della informazioni è quello di gar generare molteplici volte una stessa risposta e poi verificarne la consistenza. 

In [201]:
model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 1,"stop_sequences": ["\nQ:"]} ## Aumento la temperature per esplorare path diversi
    )

In [202]:
template_1 = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.
Q: {text}
A:"""


template_2 = """Given the answer to the following question '{text}' and the following provided answers, verify the consistency and give me the final correct answer. If necessary use voting.
Answer 1: {ans_1},
Answer 2: {ans_2}, 
Answer 3: {ans_3}
Think step by step
"""


In [203]:
prompt_1 = ChatPromptTemplate.from_template(template=template_1)
chain_1 = prompt_1 | model 

prompt_2 = ChatPromptTemplate.from_template(template=template_2)
chain_2 = prompt_2 | model 

In [206]:
text = "When I was 6 my sister was half my age plus the age of my twin brother divided by 3. Now I’m 70 how old is my sister?"

In [207]:
output = chain_1.batch([{"text":text},{"text":text},{"text":text}])
chain_2.invoke({"text":text,"ans_1":output[0],"ans_2":output[1],"ans_3":output[2]})

' Based on the provided explanations, Answer 1, Answer 2, and Answer 3 are all consistent and arrive at the same conclusion - that the age of my sister is now 69. Since all three answers agree, I do not need additional voting and can confirm that the age of my sister is 69.'